In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("SparkSQL basic example") \
  .enableHiveSupport() \
  .getOrCreate()

## 10.3 스파크 SQL
- 스파크 SQL은 하이브 메타스토어를 사용하므로 하이브와 잘 연동됨
  - 하이브 버전 설정 : spark.sql.hive.metastore.version
  - 초기화 방식 변경 : spark.sql.hive.metastore.jars
  - 클래스 접두사 설정 : spark.sql.hive.metastore.sharedPrefixes

## 10.4 스파크 쿼리 실행 방법
- 스파크 SQL CLI
  ```sh
  ./bin/spark-sq;
  ```
- 스파크 프로그래밍 SQL 인터페이스
  ```sh
  spark.sql("SELECT 1+1").show()
  ```
- 스파크 SQL 쓰리프트 JDBC/ODBC 서버
  - 자바 데이터베이스 연결 인터페이스 제공
  - 하이브 1.2.1 버전의 HiveServer2에 맞추어 구현

In [4]:
display(dbutils.fs.ls("/databricks-datasets/definitive-guide/data/flight-data/json/"))

path,name,size
dbfs:/databricks-datasets/definitive-guide/data/flight-data/json/2010-summary.json,2010-summary.json,21353
dbfs:/databricks-datasets/definitive-guide/data/flight-data/json/2011-summary.json,2011-summary.json,21301
dbfs:/databricks-datasets/definitive-guide/data/flight-data/json/2012-summary.json,2012-summary.json,20529
dbfs:/databricks-datasets/definitive-guide/data/flight-data/json/2013-summary.json,2013-summary.json,20968
dbfs:/databricks-datasets/definitive-guide/data/flight-data/json/2014-summary.json,2014-summary.json,20177
dbfs:/databricks-datasets/definitive-guide/data/flight-data/json/2015-summary.json,2015-summary.json,21368


In [5]:
# View 테이블을 생성
spark.read.json("/databricks-datasets/definitive-guide/data/flight-data/json/2015-summary.json").createOrReplaceTempView("flights")

display(spark.sql(
"""
SELECT DEST_COUNTRY_NAME, sum(COUNT)
FROM flights GROUP BY DEST_COUNTRY_NAME limit 20
"""
))

DEST_COUNTRY_NAME,sum(COUNT)
Anguilla,41
Russia,176
Paraguay,60
Senegal,40
Sweden,118
Kiribati,26
Guyana,64
Philippines,134
Djibouti,1
Malaysia,2


## 10.5 카탈로그
- 스파크의 가장 높은 수상화 단계
- 테이블, 데이터베이스, 함수를 조회하는 등 여러 가지 유용한 함수를 제공

## 10.6 테이블
- DataFrame과 논리적으로 동일
- 스파크에서 테이블을 생성하면 default 데이터베이스에 등록됨
- 임시 테이블 개념이 없으며 데이터를 가지지 않은 뷰만 존재
- 관리형 테이블과 외부 테이블이 있으며 디스크에 저장된 파일을 이용해 테이블을 저으이하면 외브 테이블을 활용하는 것임

### 10.6.2 테이블 생성하기
- USING을 통해 포맷을 지정하지 않으면 스파크는 기본적으로 하이브 SerDe 설정을 사용해 Reader, Writer 성능에 악영향을 줌

In [8]:
spark.sql(
"""
CREATE TABLE IF NOT EXISTS flights2(
  DEST_COUNTRY_NAME STRING,
  ORIGIN_COUNTRY_NAME STRING,
  count LONG
) USING JSON LOCATION '/ml/sparksql/jsonsample'
"""
)

Out[8]: DataFrame[]

In [9]:
%sql
show tables

database,tableName,isTemporary
default,flights2,false
,flights,true


### CTAS 패턴으로 테이블 생성

In [11]:
# 특정 테이블에서 원하는 데이터만 추출해서 새로운 테이블을 생성
# CTAS 패턴

spark.sql(
"""
CREATE TABLE IF NOT EXISTS flights_from_select
USING Parquet AS SELECT * FROM flights
"""
)

Out[9]: DataFrame[]

In [12]:
%sql
show tables

database,tableName,isTemporary
default,flights2,false
default,flights_from_select,false
,flights,true


In [13]:
%sql
select * from flights_from_select limit 10;

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


### 10.6.3 외부 테이블 생성하기
- 외부 테이블의 메타데이터를 관리하지만 데이터 파일은 스파크에서 관리하지 않음

In [15]:
spark.sql(
"""
CREATE EXTERNAL TABLE IF NOT EXISTS hive_flights(
  DEST_COUNTRY_NAME STRING,
  ORIGIN_COUNTRY_NAME STRING,
  count LONG
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '.'
LOCATION '/ml/flight-data-hive'
"""
)

Out[10]: DataFrame[]

In [16]:
spark.sql(
"""
CREATE EXTERNAL TABLE hive_flight2
ROW FORMAT DELIMITED FIELDS TERMINATED BY '.'
LOCATION './ml/flight-data-hive' AS SELECT * FROM flights
"""
)

Out[11]: DataFrame[]

In [17]:
%sql
select * from hive_flight2 limit 10;

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


In [18]:
display(dbutils.fs.ls("./ml/flight-data-hive/")) # 데이터가 저장된 것을 확인할 수 있음

path,name,size
dbfs:/ml/flight-data-hive/_SUCCESS,_SUCCESS,0
dbfs:/ml/flight-data-hive/_committed_718921104859482133,_committed_718921104859482133,108
dbfs:/ml/flight-data-hive/_started_718921104859482133,_started_718921104859482133,0
dbfs:/ml/flight-data-hive/part-00000-tid-718921104859482133-697c2007-23f0-4cab-be4a-7e8cdda8e34b-356-1-c000,part-00000-tid-718921104859482133-697c2007-23f0-4cab-be4a-7e8cdda8e34b-356-1-c000,7032


### 10.6.4 테이블에 데이터 삽입하기

In [20]:
%sql
describe flights_from_select

col_name,data_type,comment
DEST_COUNTRY_NAME,string,null
ORIGIN_COUNTRY_NAME,string,null
count,bigint,null


In [21]:
%sql
SELECT count(DEST_COUNTRY_NAME) FROM flights_from_select

count(DEST_COUNTRY_NAME)
256


In [22]:
spark.sql(
"""
INSERT INTO flights_from_select
SELECT * FROM flights LIMIT 20
"""
)

Out[16]: DataFrame[]

In [23]:
%sql
select count(DEST_COUNTRY_NAME) FROM flights_from_select

count(DEST_COUNTRY_NAME)
276


In [24]:
# 메타 데이터 혹인
display(spark.sql(
"""
DESCRIBE TABLE flights
"""
))

col_name,data_type,comment
DEST_COUNTRY_NAME,string,null
ORIGIN_COUNTRY_NAME,string,null
count,bigint,null


### 기타 명령어
- DESCRIBE : 메타정보 확인
- SHOW PARTITIONS : 파티셔닝 스키마 정보 확인
- REFRESH : 캐싱된 항목을 갱신
- MSCK REPAIR TABLE : 관리하는 테이블의 파티션 정보를 새로고침
- DROP TABLE : 외부 테이블을 제거하면 데이터는 삭제되지 않지만, 외부 테이블명을 이용해 데이터 조회는 안됨
- CACHE/UNCACHE TABLE

### 10.10 고급 주제
- 구조체, 리스트, 맵 세가지 타입이 존재

### 리스트
- 값의 리스트를 만드는 collect_list, 중복값을 제거하는 collect_set이 있음

In [27]:
%sql
select * from flights

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


In [28]:
%sql
select dest_country_name, collect_list(count) as flights_counts_cl, sum(count) as flights_counts, collect_set(origin_country_name) as origin_set, origin_country_name
from flights
group by dest_country_name, origin_country_name
limit 20;

dest_country_name,flights_counts_cl,flgiths_counts,origin_set,origin_country_name
Croatia,List(2),2,List(United States),United States
Kosovo,List(1),1,List(United States),United States
Romania,List(14),14,List(United States),United States
Ireland,List(335),335,List(United States),United States
United States,List(12),12,List(Egypt),Egypt
India,List(61),61,List(United States),United States
Niger,List(2),2,List(United States),United States
Singapore,List(3),3,List(United States),United States
Grenada,List(53),53,List(United States),United States
United States,List(608),608,List(Costa Rica),Costa Rica


In [29]:
sql = """
SELECT DEST_COUNTRY_NAME, ARRAY(1, 2, 3) FROM flights
"""

display(spark.sql(sql).limit(10))

DEST_COUNTRY_NAME,"array(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
Egypt,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
Costa Rica,"List(1, 2, 3)"
Senegal,"List(1, 2, 3)"
Moldova,"List(1, 2, 3)"


In [30]:
%sql
-- 위치 인덱싱
select dest_country_name, array(1, 2, 3) ,array(1, 2, 3)[0] from flights limit 10;

dest_country_name,"array(1, 2, 3)","array(1, 2, 3)[0]"
United States,"List(1, 2, 3)",1
United States,"List(1, 2, 3)",1
United States,"List(1, 2, 3)",1
Egypt,"List(1, 2, 3)",1
United States,"List(1, 2, 3)",1
United States,"List(1, 2, 3)",1
United States,"List(1, 2, 3)",1
Costa Rica,"List(1, 2, 3)",1
Senegal,"List(1, 2, 3)",1
Moldova,"List(1, 2, 3)",1


In [31]:
%sql
CREATE OR REPLACE TEMP VIEW flights_agg 
AS SELECT DEST_COUNTRY_NAME, collect_list(count) as flight_counts
FROM flights GROUP BY DEST_COUNTRY_NAME

In [32]:
%sql
show tables

database,tableName,isTemporary
default,flights2,false
default,flights_from_select,false
default,hive_flight2,false
default,hive_flights,false
,flights,true
,flights_agg,true


In [33]:
display(spark.sql("""
SELECT * FROM flights_agg limit 10
"""))

DEST_COUNTRY_NAME,flight_counts
Anguilla,List(41)
Russia,List(176)
Paraguay,List(60)
Senegal,List(40)
Sweden,List(118)
Kiribati,List(26)
Guyana,List(64)
Philippines,List(134)
Djibouti,List(1)
Malaysia,List(2)


In [34]:
# 반대로 동작
sql = """
SELECT explode(flight_counts) as count, DEST_COUNTRY_NAME FROM flights_agg
"""

display(spark.sql(sql).limit(10))

count,DEST_COUNTRY_NAME
41,Anguilla
176,Russia
60,Paraguay
40,Senegal
118,Sweden
26,Kiribati
64,Guyana
134,Philippines
1,Djibouti
2,Malaysia


### 10.10.2 함수
- SHOW FUNCTIONS
- SHOW SYSTEM FUNCTIONS
- SHOW USER FUNCTIONS
- SHOW FUNCTIONS "S"
- SHOW FUNCTIONS LIKE "collect"
- 사용자 정의 함수

In [36]:
def power3(num):
  return num * num * num

spark.udf.register("power_3", f=power3)

sql = "SELECT count, power_3(count) FROM flights"
display(spark.sql(sql).limit(10))

count,power_3(count)
15,3375
1,1
344,40707584
15,3375
62,238328
1,1
62,238328
588,203297472
40,64000
1,1


### 10.10.3 서브쿼리
- 비상호 연결쿼리 : 서브쿼리와 연관된 정보를 사용하지 않음
- 상호 연결쿼리 : 내부쿼리가 외부 쿼리의 결과를 참조

### 비상호 연관쿼리

In [39]:
subsql = """
SELECT DEST_COUNTRY_NAME FROM flights
GROUP BY DEST_COUNTRY_NAME ORDER BY sum(COUNT) DESC LIMIT 5
"""
display(spark.sql(subsql))

DEST_COUNTRY_NAME
United States
Canada
Mexico
United Kingdom
Japan


In [40]:
sql = """
SELECT * FROM flights
WHERE ORIGIN_COUNTRY_NAME IN
(SELECT DEST_COUNTRY_NAME FROM flights
GROUP BY DEST_COUNTRY_NAME ORDER BY sum(count) DESC LIMIT 5)
"""

display(spark.sql(sql).limit(20))

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
Egypt,United States,15
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1
Guyana,United States,64
Malta,United States,1
Anguilla,United States,41
Bolivia,United States,30
Algeria,United States,4
Turks and Caicos Islands,United States,230


### 상호 연관쿼리
- 목적지 국가에서 되돌아올 수 있는 항공편이 있는지 알고 싶다면 목적지 국가를 출발지 국가로, 출발지 구가를 목적지 국가로 설정하여 항공편이 있는지 확인

In [42]:
%sql
SELECT * FROM flights f1
WHERE EXISTS (SELECT 1 FROM flights f2
    WHERE f1.dest_country_name = f2.origin_country_name)
AND EXISTS (SELECT 1 FROM flights f2
    WHERE f2.dest_country_name = f1.origin_country_name)
limit 20

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
United States,Sint Maarten,325
